In [1]:
import numpy as np
from scipy.spatial.transform import Rotation
import itertools
from collections import Counter


# Get inputs

In [2]:
# with open("test_input.txt") as f:
with open("input.txt") as f:
    inp = f.readlines()
    inp = [line.rstrip() for line in inp]
#inp

In [3]:
scan_dict = {}
for line in inp:
    if line == '':
        continue
    if line[1] == '-':
        cur_scanner = int(line.lstrip('--- scanner ').rstrip(' ---'))
        scan_dict[cur_scanner] = []
    else:
        scan_dict[cur_scanner].append(tuple([int(x) for x in line.split(',')]))

## Q1 & Q2

In [4]:
def try_align(aligned, beacon):
    aligned_beacons = []
    scan_loc = []
    prev = prev2 = None
    for dim in range(3):
        x = [pos[dim] for pos in aligned]
        for perm in [(0,1),(1,1),(2,1),(0,-1),(1,-1),(2,-1)]:
            if perm[0] == prev or perm[0] == prev2:
                continue
            
            transformed = [pos[perm[0]]*perm[1] for pos in beacon]
            distances = [b-a for (a,b) in itertools.product(x, transformed)]
            counts = Counter(distances).most_common(1)
            if counts[0][1] >= 12:
                break
        if counts[0][1] < 12:
            return None
        prev2 = prev 
        prev = perm[0]
        aligned_beacons.append([v - counts[0][0] for v in transformed])
        scan_loc.append(counts[0][0])
        
    aligned_beacons = list(zip(aligned_beacons[0],aligned_beacons[1],aligned_beacons[2]))
    return aligned_beacons, scan_loc

In [5]:
def find_beacons(scanners):
    done = set()
    ocean = [scanners.pop(0)]
    remaining_scanners = list(scanners.values())
    scan_locs = [(0,0,0)]
    while ocean:
        aligned = ocean.pop()
        tmp = []
        for beacon in remaining_scanners:
            successful_align = try_align(aligned, beacon)
            if successful_align:
                new_ocean, scan_loc = successful_align
                scan_locs.append(scan_loc)
                ocean.append(new_ocean)
            else:
                tmp.append(beacon)
        remaining_scanners = tmp
        done.update(aligned)
        
    return done, scan_locs

In [6]:
beacons, scan_locs = find_beacons(scan_dict.copy())
print(f"Part 1: {len(beacons)}")

Part 1: 400


In [7]:
# Part 2
scan_perms = itertools.product(scan_locs,scan_locs)
largest_dist = 0
for x,y in scan_perms:
    distance = sum([abs(a-b) for a,b in zip(x,y)])
    if distance > largest_dist:
        largest_dist = distance



In [8]:
print(f"Part 2: {largest_dist}")

Part 2: 12168
